In [1]:
from datetime import datetime, timedelta

import ccxt
import numpy as np
import pandas as pd

import plotly.graph_objects as go

ex = ccxt.binance()

In [2]:
symbols = ['FORTH/BUSD',
'DOT/BUSD',
'FIL/BUSD',
'BTCST/BUSD',
'SHIB/BUSD',
'GMT/BUSD',
'ICP/BUSD',
'RUNE/BUSD',
'APE/BUSD',
'LINK/BUSD',
'GAL/BUSD',
'FTM/BUSD',
'GALA/BUSD',
'TRB/BUSD',
'WAVES/BUSD',
'PROM/BUSD',
'YFI/BUSD',
'NEAR/BUSD',
'AXS/BUSD',
'LDO/BUSD',
'BOND/BUSD',
'JASMY/BUSD',
'LUNA/BUSD',
'PEOPLE/BUSD',
'LIT/BUSD',
'FARM/BUSD',
'PORTO/BUSD',
'ONE/BUSD',
'IMX/BUSD',
'CHZ/BUSD',
'AUCTION/BUSD',
'MANA/BUSD',
'EGLD/BUSD',
'ATOM/BUSD',
'UNI/BUSD',
'BURGER/BUSD',
'SUSHI/BUSD',
'FTT/BUSD',
'CTK/BUSD',
'ZIL/BUSD',
'BEL/BUSD',
'CAKE/BUSD',
'XVS/BUSD',
'THETA/BUSD',
'DEGO/BUSD',
'STPT/BUSD',
'AR/BUSD',
'LEVER/BUSD',
'ENS/BUSD',
'SNX/BUSD',
'VET/BUSD',
'CREAM/BUSD',
'GLMR/BUSD',
'ASR/BUSD',
'WOO/BUSD',
'DYDX/BUSD',
'CRV/BUSD',
'KDA/BUSD',
'ALGO/BUSD',
'AAVE/BUSD',
'CHESS/BUSD',
'BCH/BUSD',
'DAR/BUSD',
'JUV/BUSD',
'ENJ/BUSD',
'GRT/BUSD',
'BAKE/BUSD',
'EOS/BUSD',
'ALCX/BUSD',
'ALPINE/BUSD',
'DODO/BUSD',
'OG/BUSD',
'RSR/BUSD',
'LAZIO/BUSD',
'KP3R/BUSD',
'SLP/BUSD',
'ALICE/BUSD',
'ANC/BUSD',
'TLM/BUSD',
'PYR/BUSD',
'LRC/BUSD',
'YGG/BUSD',
'CELO/BUSD',
'FRONT/BUSD',
'MOVR/BUSD',
'RAD/BUSD',
'HIGH/BUSD',
'RVN/BUSD',
'MINA/BUSD',
'CITY/BUSD',
'OGN/BUSD',
'BADGER/BUSD',
'FIRO/BUSD',
'PSG/BUSD',
'MTL/BUSD',
'BICO/BUSD',
'HNT/BUSD',
'LPT/BUSD',
'AUDIO/BUSD',
'FLOW/BUSD',
'VIDT/BUSD',
'XTZ/BUSD',
'ZRX/BUSD',
'SUPER/BUSD',
'XLM/BUSD',
'QI/BUSD',
'ATA/BUSD',
'TWT/BUSD',
'UFT/BUSD',
'BETA/BUSD',
'DOCK/BUSD',
'COMP/BUSD',
'BAND/BUSD',
'QNT/BUSD',
'POND/BUSD',
'ERN/BUSD',
'SXP/BUSD',
'NEO/BUSD',
'ACA/BUSD',
'KLAY/BUSD',
'OOKI/BUSD',
'C98/BUSD',
'JOE/BUSD',
'XMR/BUSD',
'FIDA/BUSD',
'VOXEL/BUSD',
'QUICK/BUSD',
'CVP/BUSD',
'IDEX/BUSD',
'INJ/BUSD',
'ACH/BUSD',
'KNC/BUSD',
'MASK/BUSD',
'API3/BUSD',
'CTXC/BUSD',
'MBOX/BUSD',
'ACM/BUSD',
'1INCH/BUSD',
'DENT/BUSD',
'LOKA/BUSD',
'MIR/BUSD',
'PERP/BUSD',
'LINA/BUSD',
'HBAR/BUSD',
'FXS/BUSD',
'KEY/BUSD',
'ATM/BUSD',
'BSW/BUSD',
'ASTR/BUSD',
'ZEC/BUSD',
'CELR/BUSD',
'ALPHA/BUSD',
'FET/BUSD',
'ANT/BUSD',
'STRAX/BUSD',
'CLV/BUSD',
'SPELL/BUSD',
'SYS/BUSD',
'BAR/BUSD',
'ADX/BUSD',
'COTI/BUSD',
'IOTX/BUSD',
'KAVA/BUSD',
'ICX/BUSD',
'TVK/BUSD',
'MLN/BUSD',
'TORN/BUSD',
'CVX/BUSD',
'STORJ/BUSD',
'DIA/BUSD',
'AUTO/BUSD',
'SRM/BUSD',
'EPX/BUSD',
'SCRT/BUSD',
'MKR/BUSD',
'AERGO/BUSD',
'ELF/BUSD',
'BAT/BUSD',
'AKRO/BUSD',
'IOTA/BUSD',
'XEC/BUSD',
'SKL/BUSD',
'NULS/BUSD',
'DUSK/BUSD',
'BTG/BUSD',
'WAXP/BUSD',
'UMA/BUSD',
'XNO/BUSD',
'OCEAN/BUSD',
'COCOS/BUSD',
'KSM/BUSD',
'HOT/BUSD',
'CHR/BUSD',
'RNDR/BUSD',
'REEF/BUSD',
'IQ/BUSD',
'NMR/BUSD',
'DNT/BUSD',
'TRIBE/BUSD',
'SANTOS/BUSD',
'ILV/BUSD',
'TROY/BUSD',
'BIFI/BUSD',
'REQ/BUSD',
'GTC/BUSD',
'MDT/BUSD',
'SFP/BUSD',
'BAL/BUSD',
'ORN/BUSD',
'TOMO/BUSD',
'CTSI/BUSD',
'AVA/BUSD',
'AGLD/BUSD',
'OMG/BUSD',
'BNX/BUSD',
'TKO/BUSD',
'ARPA/BUSD',
'IOST/BUSD',
'WIN/BUSD',
'REI/BUSD',
'BLZ/BUSD',
'DEXE/BUSD',
'MOB/BUSD',
'FOR/BUSD',
'MC/BUSD',
'WRX/BUSD',
'MBL/BUSD',
'FLUX/BUSD',
'HARD/BUSD',
'FIO/BUSD',
'LTO/BUSD',
'POLS/BUSD',
'ONT/BUSD',
'DREP/BUSD',
'QTUM/BUSD',
'DF/BUSD',
'GTO/BUSD',
'ALPACA/BUSD',
'SUN/BUSD',
'PHA/BUSD',
'DATA/BUSD',
'DASH/BUSD',
'ANKR/BUSD',
'FIS/BUSD',
'NEXO/BUSD',
'STX/BUSD',
'CFX/BUSD',
'BNT/BUSD',
'OM/BUSD',
'RAY/BUSD',
'RARE/BUSD']

In [3]:
symbols_truncated = symbols[:10]

In [4]:
def get_ohlcv(symbol, days_count, timeframe):
    from_ts = int((datetime.today() - timedelta(days=days_count)).timestamp() * 1000)

    ts = []
    time = []
    open = []
    high = []
    low = []
    close = []
    volume = []

    mid = []
    open_close_dif = []
    low_high_dif = []
    open_close_dif_abs = []
    low_high_dif_abs = []

    ohlcvs = ex.fetch_ohlcv(symbol, timeframe, since=from_ts)

    for ohlcv in ohlcvs:
        t, o, h, l, c, v = ohlcv
        ts.append(t)
        time.append(pd.to_datetime(t, unit='ms'))
        open.append(o)
        high.append(h)
        low.append(l)
        close.append(c)
        volume.append(v)

        mid.append((h+l) / 2)
        open_close_dif.append(o-c)
        low_high_dif.append(l-h)
        open_close_dif_abs.append(abs(o-c))
        low_high_dif_abs.append(abs(l-h))

    return pd.DataFrame({"ts": ts, "time": time, "open": open, "high": high, "low": low, "close": close, "volume": volume, "mid": mid, "open_close_dif": open_close_dif, "low_high_dif": low_high_dif, "open_close_dif_abs": open_close_dif_abs, "low_high_dif_abs": low_high_dif_abs})

In [5]:
def fill_ohcls(ohcls, log_price = True):
    diff = np.abs(np.diff(ohcls['mid']))

    mid_shifted = ohcls['mid'][1:].to_numpy()
    mid_without_last = ohcls['mid'][:-1].to_numpy()

    if log_price:
        div = mid_shifted / mid_without_last
        log_diff = np.log(div)
    else:
        div = (mid_shifted - mid_without_last) / mid_shifted
        log_diff = div

    ohcls.loc[1:, ['diff']] = diff
    ohcls.loc[1:, ['log_diff']] = np.abs(log_diff)

    return ohcls[1:]

In [6]:
def plot_volatility(ohcls):
    volatility = go.Scatter(x=ohcls['time'], y=ohcls["log_diff"], name = 'price_diff', mode='lines')

    time_period = ohcls.iloc[[0,-1]]['time']
    mean = ohcls['log_diff'].mean()
    y_sigma1 = ohcls['log_diff'].std()
    y_sigma2 = y_sigma1 * 2
    y_sigma3 = y_sigma1 * 3

    mean = go.Scatter(x=time_period, y = [mean, mean], name = 'mean', mode='lines', line={'width': 1})
    sigma1 = go.Scatter(x=time_period, y = [y_sigma1, y_sigma1], name = 'sigma1', mode='lines', line={'width': 1})
    sigma2 = go.Scatter(x=time_period, y = [y_sigma2, y_sigma2], name = 'sigma2', mode='lines', line={'width': 1})
    sigma3 = go.Scatter(x=time_period, y = [y_sigma3, y_sigma3], name = 'sigma3', mode='lines', line={'width': 1})

    fig = go.Figure()
    fig.add_trace(volatility)
    fig.add_trace(mean)
    fig.add_trace(sigma1)
    fig.add_trace(sigma2)
    fig.add_trace(sigma3)
    fig.show()

In [7]:
def annualize(symbol, ohcls):
    vol = ohcls['log_diff'].std()

    timestamps = ohcls['ts'].to_numpy()
    ts_start, ts_end = timestamps[0], timestamps[-1]
    dt_ms = ts_end - ts_start
    year_ms = 365 * 24 * 60 * 60 * 1000

    n_periods = year_ms / dt_ms
    annualized = vol * np.sqrt(n_periods)

    # print(symbol, 'period:', int((dt_ms / 1000 / 60 / 60 / 24) * 100)/100, 'days')

    return annualized


In [8]:
def create_ohlcv_df(symbol, days_count, timeframe, log_price = True):
    """
    start volatility investigation
    :param str symbol: Unified CCXT market symbol
    :param int days_count: last days_count to fetch
    :param str timeframe: "5m","15m","30m","1h","2h","4h","6h","12h", or "1d"
    :param bool|None log_price: default True: use log(price_diff) or not
    """
    ohcls = get_ohlcv(symbol, days_count, timeframe)
    ohcls = fill_ohcls(ohcls, log_price)

    return ohcls

In [14]:
def start(symbols, days_count, timeframe):
    coins = []
    volatility = []

    for symbol in symbols:
        ohcls = create_ohlcv_df(symbol, days_count, timeframe)
        annualized_vol = annualize(symbol, ohcls)

        coins.append(symbol)
        volatility.append(annualized_vol)

    print(pd.DataFrame({"annual_vol": volatility}, index=coins))

In [15]:
start(symbols_truncated, 5, '15m')

            annual_vol
FORTH/BUSD    0.021063
DOT/BUSD      0.011066
FIL/BUSD      0.014091
BTCST/BUSD    0.060955
SHIB/BUSD     0.013859
GMT/BUSD      0.013025
ICP/BUSD      0.012502
RUNE/BUSD     0.014872
APE/BUSD      0.016067
LINK/BUSD     0.014998
